In [5]:
from google.colab import files
from scapy.all import rdpcap, DNS, DNSQR
import datetime

# Step 1: Upload the pcap file
print("👉 Please upload your 8.pcap file")
uploaded = files.upload()

# Step 2: Path to uploaded file
PCAP_FILE = list(uploaded.keys())[0]  # automatically picks the uploaded file

def build_custom_header(seq_id: int) -> str:
    """Builds 8-byte custom header in format HHMMSSID"""
    now = datetime.datetime.now()
    header = f"{now.hour:02d}{now.minute:02d}{now.second:02d}{seq_id:02d}"
    return header

def main():
    # Read pcap
    packets = rdpcap(PCAP_FILE)

    # Filter DNS query packets
    dns_queries = [pkt for pkt in packets if pkt.haslayer(DNS) and pkt[DNS].qd is not None and pkt[DNS].qr == 0]

    print(f"✅ Found {len(dns_queries)} DNS query packets\n")

    # Print each query with custom header
    for i, pkt in enumerate(dns_queries):
        query_name = pkt[DNSQR].qname.decode(errors="ignore")
        custom_header = build_custom_header(i)
        print(f"Query: {query_name}, Custom Header: {custom_header}")

if __name__ == "__main__":
    main()


👉 Please upload your 8.pcap file


Saving 8.pcap to 8.pcap
✅ Found 23 DNS query packets

Query: _apple-mobdev._tcp.local., Custom Header: 14270300
Query: _apple-mobdev._tcp.local., Custom Header: 14270301
Query: github.com., Custom Header: 14270302
Query: Brother MFC-7860DW._pdl-datastream._tcp.local., Custom Header: 14270303
Query: Brother MFC-7860DW._pdl-datastream._tcp.local., Custom Header: 14270304
Query: bing.com., Custom Header: 14270305
Query: facebook.com., Custom Header: 14270306
Query: Brother MFC-7860DW._pdl-datastream._tcp.local., Custom Header: 14270307
Query: Brother MFC-7860DW._pdl-datastream._tcp.local., Custom Header: 14270308
Query: amazon.com., Custom Header: 14270309
Query: _apple-mobdev._tcp.local., Custom Header: 14270310
Query: Brother MFC-7860DW._pdl-datastream._tcp.local., Custom Header: 14270311
Query: Brother MFC-7860DW._pdl-datastream._tcp.local., Custom Header: 14270312
Query: linkedin.com., Custom Header: 14270313
Query: Brother MFC-7860DW._pdl-datastream._tcp.local., Custom Header: 142703

In [10]:
from scapy.all import PcapReader, DNS, DNSQR
import datetime
import pandas as pd

PCAP_FILE = "8.pcap"

def build_custom_header(seq_id: int) -> str:
    now = datetime.datetime.now()
    return f"{now.hour:02d}{now.minute:02d}{now.second:02d}{seq_id:02d}"

# Define fake resolution rules
resolution_rules = {
    "github.com.": "140.82.113.3",
    "bing.com.": "204.79.197.200",
    "facebook.com.": "157.240.22.35",
    "amazon.com.": "176.32.103.205",
    "linkedin.com.": "108.174.10.10",
    "stackoverflow.com.": "151.101.193.69",
    "_apple-mobdev._tcp.local.": "224.0.0.251",
    "Brother MFC-7860DW._pdl-datastream._tcp.local.": "192.168.1.50",
}

report = []
seq = 0

# Use PcapReader (streams packets instead of loading whole file)
with PcapReader(PCAP_FILE) as pcap:
    for pkt in pcap:
        if pkt.haslayer(DNS) and pkt[DNS].qd is not None and pkt[DNS].qr == 0:
            query_name = pkt[DNSQR].qname.decode(errors="ignore")
            header = build_custom_header(seq)
            resolved_ip = resolution_rules.get(query_name, "0.0.0.0")
            report.append((query_name, header, resolved_ip))
            seq += 1
        if seq >= 30:  # stop after 30 queries for speed (adjust if needed)
            break

# Print report
print("Final Report Table:\n")
print(f"{'Query':40} | {'Custom Header':12} | {'Resolved IP'}")
print("-"*70)
for row in report:
    print(f"{row[0]:40} | {row[1]:12} | {row[2]}")

# Save report
df = pd.DataFrame(report, columns=["Query", "Custom Header", "Resolved IP"])
df.to_csv("dns_report.csv", index=False)
print("\n📂 Report saved as dns_report.csv")


Final Report Table:

Query                                    | Custom Header | Resolved IP
----------------------------------------------------------------------
_apple-mobdev._tcp.local.                | 15384900     | 224.0.0.251
_apple-mobdev._tcp.local.                | 15384901     | 224.0.0.251
github.com.                              | 15391002     | 140.82.113.3
Brother MFC-7860DW._pdl-datastream._tcp.local. | 15392303     | 192.168.1.50
Brother MFC-7860DW._pdl-datastream._tcp.local. | 15392404     | 192.168.1.50
bing.com.                                | 15392905     | 204.79.197.200
facebook.com.                            | 15395606     | 157.240.22.35
Brother MFC-7860DW._pdl-datastream._tcp.local. | 15401107     | 192.168.1.50
Brother MFC-7860DW._pdl-datastream._tcp.local. | 15401208     | 192.168.1.50
amazon.com.                              | 15403709     | 176.32.103.205
_apple-mobdev._tcp.local.                | 15405810     | 224.0.0.251
Brother MFC-7860DW._pdl-datast

In [12]:
from google.colab import files
files.download("dns_report.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
from google.colab import files
# files.download("filename.py")   # code file
files.download("dns_report.csv")    # CSV file

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
!pip install scapy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 60.6 MB/s eta 0:00:00
